In [7]:
import Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("GLPK")

    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`


In [123]:
using JuMP
using HiGHS
using GLPK

model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

## Variables
Variables ***VEG1, VEG2, OIL1, OIL2 and OIL3*** are represented by columns in the matrixes

Periods / months are represented by rows in the matrixes

Varible ***PROD*** represents the quantity of production that should be made

In [124]:
price = 150
init_store = 500
target_store = 500
veg_cap = 200
oil_cap = 250

min_hardness = 3
max_hardness = 6
holding_cost = 5

hardnes = [8.8, 6.1, 2.0, 4.2, 5.0]

@variable(model, produce[1:6] >= 0)
@variable(model, bought[1:6,1:5] >= 0)
@variable(model, used[1:6,1:5] >= 0)
@variable(model, stored[1:6,1:5] >= 0)


6×5 Matrix{VariableRef}:
 stored[1,1]  stored[1,2]  stored[1,3]  stored[1,4]  stored[1,5]
 stored[2,1]  stored[2,2]  stored[2,3]  stored[2,4]  stored[2,5]
 stored[3,1]  stored[3,2]  stored[3,3]  stored[3,4]  stored[3,5]
 stored[4,1]  stored[4,2]  stored[4,3]  stored[4,4]  stored[4,5]
 stored[5,1]  stored[5,2]  stored[5,3]  stored[5,4]  stored[5,5]
 stored[6,1]  stored[6,2]  stored[6,3]  stored[6,4]  stored[6,5]

In [125]:
cost = [[110,120,130,110,115] [130,130,110,90,115] [110,140,130,100,95] [120,110,120,120,125] [100,120,150,110,105] [90,100,140,80,135]]'

6×5 adjoint(::Matrix{Int64}) with eltype Int64:
 110  120  130  110  115
 130  130  110   90  115
 110  140  130  100   95
 120  110  120  120  125
 100  120  150  110  105
  90  100  140   80  135

## Constraints

Initial Balance:

In [126]:
for i in 1:5
    @constraint(model, bought[1,i] - used[1,i] - stored[1,i] == init_store)
end

Balance

In [127]:
for i in 2:6
    for j in 1:5
        @constraint(model, stored[i-1,j] + bought[i,j] - used[i,j] - stored[i,j] == 0)
    end
end

Inventory Target

In [128]:
for i in 1:5
    @constraint(model, stored[6,i] == target_store)
end

Refinement Capacity

In [129]:
for i in 1:6
    @constraint(model, sum(used[i,j] for j in 1:2) <= veg_cap)
end

for i in 1:6
    @constraint(model, sum(used[i,j] for j in 3:5) <= oil_cap)
end

Hardness

In [130]:
for i in 1:6
    @constraint(model, sum(hardnes[j] * used[i,j] for j in 1:5) - max_hardness * produce[i] <= 0)
    @constraint(model, min_hardness * produce[i] - sum(hardnes[j] * used[i,j] for j in 1:5) <= 0)
end

Mass Conservation

In [131]:
for i in 1:6
    @constraint(model, sum(used[i,j] for j in 1:5) - produce[i] == 0)
end

## Objective Function
Max the total profit

In [132]:
@objective(model, Max, sum(price*produce[i] for i in 1:6) - sum(sum(cost[i,j] * used[i,j] + holding_cost * stored[i,j] for j in 1:5) for i in 1:6))

150 produce[1] + 150 produce[2] + 150 produce[3] + 150 produce[4] + 150 produce[5] + 150 produce[6] - 110 used[1,1] - 5 stored[1,1] - 120 used[1,2] - 5 stored[1,2] - 130 used[1,3] - 5 stored[1,3] - 110 used[1,4] - 5 stored[1,4] - 115 used[1,5] - 5 stored[1,5] - 130 used[2,1] - 5 stored[2,1] - 130 used[2,2] - 5 stored[2,2] - 110 used[2,3] - 5 stored[2,3] - 90 used[2,4] - 5 stored[2,4] - 115 used[2,5] - 5 stored[2,5] - 110 used[3,1] - 5 stored[3,1] - 140 used[3,2] - 5 stored[3,2] - 130 used[3,3] - 5 stored[3,3] - 100 used[3,4] - 5 stored[3,4] - 95 used[3,5] - 5 stored[3,5] - 120 used[4,1] - 5 stored[4,1] - 110 used[4,2] - 5 stored[4,2] - 120 used[4,3] - 5 stored[4,3] - 120 used[4,4] - 5 stored[4,4] - 125 used[4,5] - 5 stored[4,5] - 100 used[5,1] - 5 stored[5,1] - 120 used[5,2] - 5 stored[5,2] - 150 used[5,3] - 5 stored[5,3] - 110 used[5,4] - 5 stored[5,4] - 105 used[5,5] - 5 stored[5,5] - 90 used[6,1] - 5 stored[6,1] - 100 used[6,2] - 5 stored[6,2] - 140 used[6,3] - 5 stored[6,3] - 80 us

In [133]:
print(model)

In [134]:
optimize!(model)

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
55 rows, 81 cols, 228 nonzeros
44 rows, 70 cols, 165 nonzeros
34 rows, 50 cols, 125 nonzeros
29 rows, 40 cols, 105 nonzeros
24 rows, 30 cols, 90 nonzeros
Presolve : Reductions: rows 24(-41); columns 30(-66); elements 90(-168)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -7.8994068588e+02 Ph1: 24(70.25); Du: 29(789.941) 0s
         31    -1.0714814815e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 31
Objective value     :  1.0714814815e+05
HiGHS run time      :          0.00


In [135]:
objective_value(model)

107148.14814814816